In [1]:
# All Imports
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from amazon.api import AmazonAPI
import io
import re
from amazon_scraper import AmazonScraper
from nltk.corpus import sentiwordnet as swn

%matplotlib inline

In [2]:
# This method is done for you. Make sure to put your credentials in amazon.cfg.
def get_amazon(config_file):
    """ Read the config_file and construct an instance of AmazonAPI.
    Args:
      config_file ... A config file in ConfigParser format with Amazon credentials
    Returns:
      An instance of AmazonAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    amazon = AmazonScraper(
                   config.get('amazon', 'AMAZON_ACCESS_KEY'),
                   config.get('amazon', 'AMAZON_SECRET_KEY'),
                   config.get('amazon', 'AMAZON_ASSOC_TAG'))
                   
    return amazon

amzn = get_amazon('amazon.cfg')
print('Established Amazon connection.')

Established Amazon connection.


In [ ]:
# Fetch the 100 most viewes Product IDs of keyword (i.e 'Microsoft Surface') which falls in category (i.e 'Electronics').

pid_lst = []
for q in itertools.islice(amzn.search(Keywords='Microsoft Surface', SearchIndex='Electronics'), 100):
    print(q.asin)
    pid_lst.append(q.asin)

In [ ]:
# Store all the Product IDs in a file so that we can use it in future. 

fp = open('MS_PID.txt','w')
for item in pid_lst:
    fp.write("%s\n" %item)

fp.close()

In [ ]:
# Reading the IDs from file and put it in list. 

pid = []
pfile = open('MS_PID.txt','r')
for i in pfile.read().split():
    pid.append(i)
    
pfile.close()

In [ ]:
# To check the number of Product IDs we have fetched. 

len(pid)

In [3]:
# Take one by one Product ID and search for it. 

#pd = 'B00MA7A876'
pd = 'B00VJ1P7TS'
p = amzn.lookup(ItemId=pd)

In [11]:
p.asin

'B00VJ1P7TS'

In [4]:
# Download the reviews from amazon page by page and store the id and review text in list form. 

def Get_Reviews(pid):
    
    rs = amzn.reviews(ItemId=pid)
    myReviews = []
    myRID = []

    for r in rs.full_reviews():
    
        myRID.append(r.id)
        myReviews.append(r.text)
           
        if len(myReviews) >= 100:
            break
    
    return myRID, myReviews

In [5]:
# Calling the Get_Reviews method and get the data. 


myRID, myReviews = Get_Reviews(pd)

In [6]:
len(myRID)

100

In [7]:
myReviews

[u"I just purchased this product less than 2 months ago. I've been having several problems and apparently inolving hardware. I'm in Peru now and can't find a place were the warranty works. Would like to know if anybody have a suggestion.",
 u'Fantastic product',
 u'My husband loves this!  Arrived super fast!  He uses it all the time.',
 u'I love it, the price that I bought it was exellent for all people. I recommended it for people that use this pc for navigate, emails and so much, not for people who work all time with big programs and resolution because is a little slowly in these cases. For example: I am a programmer and it has not  the best performance compared with the macbook pro. You choose what do you want and for what.',
 u"MacBook Air , Apple is Apple , you pay for the name . Overall this computer is fairly good . I haven't had any virus' no download problems and no problems with the webcam . I do like that you can use the touch pad to scroll through everything and display thr

In [8]:
# To store the reviews in text file and named as it's Product ID. 

def Review_Store_File(p,myRID,myReviews):
    
    review_file = open('Reviews_Data/' + p.asin + '.txt','a')
    for i in range(len(myRID)):

        review_file.write(myRID[i] + "->" + myReviews[i].replace("\n", " ").encode('utf-8') + "\n")


    review_file.close()

In [9]:
Review_Store_File(p,myRID,myReviews)

In [10]:
# To store the Item into items.txt to keep track of all items.


def Item_Store_File(p):

    items_file = open('items.txt',"a")
    items_file.write(p.asin + "\t" + p.title.encode('utf-8') + "\t" + p.manufacturer + "\n")

    items_file.close()


In [12]:
Item_Store_File(p)

In [12]:
# To see the list of Items stored in 'items.txt' file. Here 3 columns are there. first depicts the Product IDs, 
# second gives Product titles and last column gives Manufacturer details.  

def Items(itm):

    itm_file = open(itm,"r")
    print itm_file.read()
    itm_file.close()


In [13]:
Items('items.txt')

B00THEP15O	Dell Inspiron 15.6 Inch Laptop	Dell Computers
B00ZOO1RFO	Dell Inspiron i3543 15.6-Inch Touchscreen Laptop	Dell Computers
B015JVB9C2	Dell Inspiron i3000-5099SLV Touchscreen Laptop	Dell Marketing USA, LP
B010DOD2G6	Dell Inspiron 15 5000 15.6 Inch Laptop	Dell Marketing USA, LP
B00VKL0I96	Dell Latitude E6420 Premium 14.1-Inch Business Laptop	Dell Computers
B010DODAWM	Dell Inspiron 11 3000 Series 2-in-1  11.6 Inch Laptop	Dell Marketing USA, LP
B013PK8LGQ	Dell Inspiron 3000 15.6-Inches Windows 8.1 Laptop 	Dell Computers
B012BPRLO4	Dell Latitude E6400 Laptop Core 2 Duo	Dell Computers
B00YD1OQ1G	Dell Inspiron N2840 14-Inch Laptop	Dell Computers
B014B9IS1S	2015 Newest Dell Chromebook, 11.6 inch HD Non-Touch LCD	Dell Computers
B00V2KX5JA	2015 Newest Model Dell Inspiron 15 Laptop Computer	Dell Computers
B00ZS64K7K	Dell Inspiron 15 i5548-4167SLV - 15.6" Touchscreen	Dell Computers
B00KMRGF28	Dell Inspiron i3531-1200BK 16-Inch Laptop,Black	Dell Computer
B00Z1R85O0	2015 Newest Dell Inspiro